In [1]:
from make_dataloader import get_dataloader
from training_test_loops import training_loop, test_loop
import torch
from torch import nn
from torch.autograd import Function
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
import time
torch.manual_seed(222222)
np.random.seed(2222)
#[(W−K+2P)/S]+1

In [2]:
root_dir = 'C:/Users/paxso/galclass_da/gal_img_full/gal_img_full/'
csv_file = root_dir+"gal_list.csv"
domain = 'source'
batch_size = 32
data = get_dataloader(csv_file, root_dir, domain, batch_size, train_size=.90, val_size=.05, test_size=.05)
train_dataset, val_dataset, test_dataset, train_dataloader, val_dataloader, test_dataloader = data

In [3]:
bf = 16
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=bf, kernel_size=3, stride=(1,1), padding='same')
        self.bn1 = nn.BatchNorm2d(bf)
        self.conv2 = nn.Conv2d(in_channels=bf, out_channels=bf*2, kernel_size=3, stride=(1,1), padding='same')
        self.bn2 = nn.BatchNorm2d(bf*2)
        self.conv3 = nn.Conv2d(in_channels=bf*2, out_channels=bf*4, kernel_size=3, stride=(1,1), padding='same')
        self.bn3 = nn.BatchNorm2d(bf*4)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.)
        
        self.fc1 = nn.Linear(in_features=bf*4*8*8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=4)
        
    def forward(self, x):
        x = x.view(-1, 3, 64, 64)
        
        x = self.dropout(self.pool(self.bn1(F.relu(self.conv1(x)))))
        x = self.dropout(self.pool(self.bn2(F.relu(self.conv2(x)))))
        x = self.dropout(self.pool(self.bn3(F.relu(self.conv3(x)))))
        #x = self.pool(self.bn4(F.relu(self.conv4(x))))
        
        x = x.view(-1, bf*4*8*8)
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        x = x.view(-1, 4)
        return x

model = NeuralNetwork().cuda()

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)
#summary(model, (3, 128, 128));

556740


In [4]:
def training_loop(train_dataloader, val_dataloader, model, optimizer, loss_fn):
    training_correct = 0.
    training_total = 0.
    training_loss = 0.
    for i, data in enumerate(train_dataloader):
        X, y = data
        #if i < 10:
            #print(X.size(), y.size())
        X, y = X.cuda(), y.reshape(-1).cuda()
        optimizer.zero_grad()
        preds = model(X)
        #if i < 10:
            #print(preds.size())
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(preds.data, 1)
        training_correct += torch.sum((predicted == y)).item()
        training_total += y.size(0)
        training_loss += loss.item()
        
    training_accuracy = training_correct / training_total
    training_loss = training_loss / len(train_dataloader)
        
    val_correct = 0.
    val_total = 0.
    val_loss = 0.    
    for data in val_dataloader:
        with torch.no_grad():
            X, y = data
            X, y = X.cuda(), y.reshape(-1).cuda()
            preds = model(X)
            loss = loss_fn(preds, y)
            _, predicted = torch.max(preds.data, 1)
            val_correct += torch.sum((predicted == y)).item()
            val_total += y.size(0)
            val_loss += loss.item()
            
    val_accuracy = val_correct / val_total
    val_loss = val_loss / len(val_dataloader)
    
    return training_accuracy, training_loss, val_accuracy, val_loss

In [5]:
epochs = 60
lr = 5.0e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss().cuda()

In [6]:
for epoch in range(epochs):
    print('epoch: ', epoch)
    start_time = time.time()
    training_accuracy, training_loss, val_accuracy, val_loss = training_loop(train_dataloader, 
                                                                         val_dataloader, model, optimizer, loss_fn)
    print(training_accuracy, training_loss, val_accuracy, val_loss)
    print(time.time() - start_time)

epoch:  0
0.6739933023561088 1.0630133333809335 0.6650107411385607 1.0697737497321527
83.39909338951111
epoch:  1
0.7214660232552936 1.0167427705267449 0.7294575725026853 1.0088145694507549
84.34377956390381
epoch:  2
0.7356369847177369 1.0034931156401987 0.7367078410311493 1.0010215570998293
87.74915790557861
epoch:  3
0.7437666415567175 0.9956390067614918 0.7521482277121375 0.9868550208505131
84.97492551803589
epoch:  4
0.7487861452747302 0.9907571110492105 0.7548335123523093 0.9843842717199367
85.61788582801819
epoch:  5
0.7540443178173736 0.9852447215867782 0.756578947368421 0.9828154112136415
87.17598342895508
epoch:  6
0.7561625036359704 0.9833826185553057 0.7540279269602578 0.9858955462091471
85.7497067451477
epoch:  7
0.7604287088762428 0.9794990197855419 0.7612781954887218 0.9786918659578577
84.20610213279724
epoch:  8
0.7629198147333249 0.9769600982455479 0.7569817400644469 0.9827854912168478
85.75131964683533
epoch:  9
0.7668727671412696 0.9732406400979845 0.7655746509129968

In [7]:
torch.save(model.state_dict(), 'C:/Users/paxso/galclass_da/80_sourceonly_network.pt')

In [8]:
def test_loop(dataloader, model, loss_fn):
    correct = 0.
    total = 0.
    all_loss = 0.    
    for data in dataloader:
        with torch.no_grad():
            X, y = data
            X, y = X.cuda(), y.reshape(-1).cuda()
            preds = model(X)
            loss = loss_fn(preds, y)
            _, predicted = torch.max(preds.data, 1)
            correct += torch.sum((predicted == y)).item()
            total += y.size(0)
            all_loss += loss.item()
            
    accuracy = correct / total
    all_loss = all_loss / len(dataloader)
    
    return accuracy, all_loss

In [9]:
test_accuracy, loss = test_loop(test_dataloader, model, loss_fn)
print(test_accuracy, loss)

0.8012889366272825 0.9407094210514183


In [10]:
model2 = NeuralNetwork().cuda()

In [11]:
test_accuracy, loss = test_loop(test_dataloader, model2, loss_fn)
print(test_accuracy, loss)

0.39339419978517726 1.371815025550613


In [12]:
#model = TheModelClass(*args, **kwargs)
model2.load_state_dict(torch.load('C:/Users/paxso/galclass_da/80_sourceonly_network.pt'))
test_accuracy, loss = test_loop(test_dataloader, model2, loss_fn)
print(test_accuracy, loss)

0.799812030075188 0.9407174551435806


In [13]:
root_dir = 'C:/Users/paxso/galclass_da/gal_img_full/gal_img_full/'
csv_file = root_dir + 'hsc_dataframe.csv'
domain = 'target'
batch_size = 32
data = get_dataloader(csv_file, root_dir, domain, batch_size, train_size=.01, val_size=.01, test_size=.98)
target_train_dataset, target_val_dataset, target_test_dataset, target_train_dataloader, target_val_dataloader, target_test_dataloader = data

In [14]:
test_accuracy, loss = test_loop(target_test_dataloader, model2, loss_fn)
print(test_accuracy, loss)

0.6166124283279095 1.1252227047882457
